In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install timm 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.3 MB/s eta 0:00:00


In [3]:
import timm
from sklearn.model_selection import StratifiedKFold

In [4]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset,random_split,SubsetRandomSampler
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter

## Dataset

In [5]:
train_data_index = pd.read_csv('../input/classify-leaves/train.csv')
test_data_index = pd.read_csv('../input/classify-leaves/test.csv')

In [6]:
categories = train_data_index['label'].unique()
categories[:10]

array(['maclura_pomifera', 'ulmus_rubra', 'broussonettia_papyrifera',
       'prunus_virginiana', 'acer_rubrum', 'cryptomeria_japonica',
       'staphylea_trifolia', 'asimina_triloba', 'diospyros_virginiana',
       'tilia_cordata'], dtype=object)

In [7]:
train_data_index['label'] = train_data_index['label'].apply(lambda x:int(np.where(categories==x)[0]))

In [8]:
train_data_index.groupby('label').size()[:10]

label
0    353
1    235
2    214
3    223
4    217
5    141
6    141
7    174
8    173
9    127
dtype: int64

In [9]:
train_data_index.iloc[0][0]

'images/0.jpg'

In [10]:
# 分层抽样 5fold
skf = StratifiedKFold(n_splits=5, random_state=80, shuffle=True)
train_index_folds,valid_index_folds = [],[]
for train_index, valid_index in skf.split(train_data_index, train_data_index['label']):
    train_index_folds.append(train_index)
    valid_index_folds.append(valid_index)
    print(f'train:{len(train_index)},valid:{len(valid_index)}')

train:14682,valid:3671
train:14682,valid:3671
train:14682,valid:3671
train:14683,valid:3670
train:14683,valid:3670


In [11]:
test_tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((224,224)),
    # You may add some transforms here.
    transforms.ColorJitter(brightness=0.5,saturation=0.3),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomHorizontalFlip(p=0.5),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [12]:
class LeavesDataset(Dataset):
    def __init__(self,data_index,tfm,root='../input/classify-leaves/',is_train=True):
        super(LeavesDataset).__init__()
        self.data_index = data_index
        self.root = root
        self.is_train = is_train
        self.tfm = tfm
    def __len__(self):
        return self.data_index.shape[0]
    def __getitem__(self,idx):
        if self.is_train:
            img_name = self.data_index.iloc[idx][0]
            label = self.data_index.iloc[idx][1]
        else:
            img_name = self.data_index.iloc[idx][0]
            label = -1
        img_path = os.path.join(self.root,img_name)
        img = Image.open(img_path)
        img_tranform = self.tfm(img)
        return img_tranform,label
    

In [13]:
def show_img(img):
    x = img
    trans = transforms.ToPILImage()
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)
    x = (x*std)+mean
    x_pil = trans(x)
    return x_pil

In [14]:
# dataset = LeavesDataset(test_data_index,test_tfm,is_train=False)
# import matplotlib.pyplot as plt
# fix, axs = plt.subplots(ncols=2, squeeze=False)
# img = transforms.ToPILImage()(dataset[1000][0])
# axs[0, 0].imshow(np.asarray(img))

## model

In [15]:
# model = models.vgg11_bn()
# model.classifier[6] = nn.Linear(in_features=4096, out_features=categories.shape[0], bias=True)
# # model

In [16]:
# def get_model():
#     model = models.resnet34(pretrained=True)
#     # model.fc = nn.Linear(in_features=4096, out_features=categories.shape[0], bias=True)
#     num_ftrs = model.fc.in_features
#     model.fc = nn.Linear(num_ftrs, out_features=176)
#     return model

In [17]:
def get_model():
    model = timm.create_model('resnet50d',pretrained=True,num_classes=176)
    return model

In [18]:
# from torchsummary import summary
# summary(model, input_size=(1,3,224,224))

## mixup

In [19]:
def mixup_data(x, y, num_classes,alpha=1.0,prob=0.1):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    batch_size = x.size()[0]
    
    lam_mix = np.random.beta(alpha, alpha,batch_size)
    lam = np.ones(batch_size, dtype=np.float32)
    lam = np.where(np.random.rand(batch_size) < prob, lam_mix.astype(np.float32), lam)
    
    
    index = torch.randperm(batch_size)

    lam = torch.tensor(lam).reshape(-1,1,1,1)
    x *= lam
    y = mixup_label(y, y[index],lam,num_classes)
    return x, y, lam

def mixup_label(y_a, y_b, lam,num_classes):
    batch_size = y_a.size()[0]
    
    label = torch.full((batch_size, num_classes), 0,dtype=lam.dtype).scatter_(1, y_a.reshape(-1,1), lam.reshape(-1,1))
    label.scatter_(1, y_b.reshape(-1,1), 1-lam.reshape(-1,1))
    
    return label

In [20]:
# x = torch.ones(4,3,3,3)
# y = torch.tensor([1,2,3,4]).reshape(-1,1)
# mixup_data(x,y,6,prob=0.5)

## train

In [21]:
lr,epoch_nums,patience = 0.001,100,20
batch_size = 64

In [22]:
model = get_model()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
criterion_test = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr = 5e-4
params_1x = [param for name, param in model.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
optimizer = torch.optim.Adam([{'params': params_1x},{'params': model.fc.parameters(),'lr': lr * 10}],lr=lr)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50d_ra2-464e36ba.pth" to /root/.cache/torch/hub/checkpoints/resnet50d_ra2-464e36ba.pth


In [23]:
train_dataset = LeavesDataset(train_data_index.iloc[train_index_folds[1]].reset_index(drop=True),
                              train_tfm)
valid_dataset = LeavesDataset(train_data_index.iloc[valid_index_folds[1]].reset_index(drop=True),
                              test_tfm)

train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                       shuffle=True,num_workers=0, pin_memory=True)
valid_iter = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, 
                                       shuffle=True,num_workers=0, pin_memory=True)

In [24]:
best_acc = 0
stale = 0

In [25]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
device = try_gpu()
print(f'device is {device}')

device is cuda:0


In [26]:
kaggle_gpu = torch.tensor([1]).cuda()
kaggle_gpu

tensor([1], device='cuda:0')

In [27]:
%load_ext tensorboard
# 可视化
writer = SummaryWriter("./runs")

In [28]:
model = model.to(device)

In [29]:
scaler = torch.cuda.amp.GradScaler() # for AMP training 
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epoch_nums*len(train_iter), eta_min=lr/20)
for epoch in range(epoch_nums):
    correct = 0
    total = 0
    train_loss = []
    model.train()
    for x,y in tqdm(train_iter):            
        inputs, targets, lam = mixup_data(x, y,num_classes=176, alpha=0.5,prob=0.1)
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs,targets)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        lr_scheduler.step()
        

#         _, predicted = torch.max(outputs.data, 1)
#         total += targets.size(0)
#         correct += lam * predicted.eq(targets_a.data).cpu().sum() + (1 - lam) * predicted.eq(targets_b.data).cpu().sum()
        
        train_loss.append(loss.item())
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = 0    #correct/total
    
    valid_acc = []
    valid_loss = []
    model.eval()
    for x,y in tqdm(valid_iter):

        with torch.no_grad():
            y_predict = model(x.to(device))
        
        loss = criterion_test(y_predict,y.to(device))
        acc = (y_predict.argmax(dim=-1) == y.to(device)).float().mean()
        
        valid_loss.append(loss.item())
        valid_acc.append(acc)
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_acc) / len(valid_acc)
    
    writer.add_scalars('loss', {'train_loss':train_loss,
                                'valid_loss':valid_loss}, epoch)
    writer.add_scalars('acc', {'train_acc':train_acc,
                               'valid_acc':valid_acc}, epoch)
    
    if valid_acc>best_acc:
        best_acc = valid_acc
        stale = 0
        # save_model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': valid_acc,
            }, f"model_best.pth.tar")
        print(f'At epoch {epoch} get better acc model[{valid_acc}]----->model_best.pth.tar')   
    else:
        stale+=1
        if stale > patience:
            print(f'At epoch {epoch} stop early because of {stale} epochs no improve')
            break
            
# save_model
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_acc': valid_acc,
    }, f"model_final_best.pth.tar")
writer.close()

  0%|          | 0/230 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

At epoch 0 get better acc model[0.7710949182510376]----->model_best.pth.tar


  0%|          | 0/230 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

...

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/230 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/230 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

In [30]:
%tensorboard --logdir ./runs

## predict

In [31]:
model = get_model()

model = model.to(device)
checkpoint = torch.load(f"model_final_best.pth.tar")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [32]:
dataset = LeavesDataset(test_data_index,test_tfm,is_train=False)

test_iter = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                       shuffle=False,num_workers=0, pin_memory=True)

In [33]:
label_list = []
model.eval()
for x,y in tqdm(test_iter):
    with torch.no_grad():
        y_predict = model(x.to(device))
    label_list += y_predict.argmax(dim=-1).reshape(-1).tolist()


  0%|          | 0/138 [00:00<?, ?it/s]

In [34]:
test_data_index['label']=categories[label_list]
test_data_index.to_csv("submission.csv",index = False)

In [35]:

checkpoint = torch.load(f"model_best.pth.tar")
model.load_state_dict(checkpoint['model_state_dict'])

test_iter = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                       shuffle=False,num_workers=0, pin_memory=True)
label_list = []
model.eval()
for x,y in tqdm(test_iter):
    with torch.no_grad():
        y_predict = model(x.to(device))
    label_list += y_predict.argmax(dim=-1).reshape(-1).tolist()
    
test_data_index.drop('label',axis=1,inplace=True)
test_data_index['label']=categories[label_list]
test_data_index.to_csv("submission_best.csv",index = False)

  0%|          | 0/138 [00:00<?, ?it/s]

In [36]:
!zip -r my.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/loss_valid_loss/ (stored 0%)
  adding: runs/loss_valid_loss/events.out.tfevents.1660907760.e7a3ae7dcd44.23.2 (deflated 57%)
  adding: runs/acc_valid_acc/ (stored 0%)
  adding: runs/acc_valid_acc/events.out.tfevents.1660907760.e7a3ae7dcd44.23.4 (deflated 58%)
  adding: runs/loss_train_loss/ (stored 0%)
  adding: runs/loss_train_loss/events.out.tfevents.1660907760.e7a3ae7dcd44.23.1 (deflated 57%)
  adding: runs/acc_train_acc/ (stored 0%)
  adding: runs/acc_train_acc/events.out.tfevents.1660907760.e7a3ae7dcd44.23.3 (deflated 65%)
  adding: runs/events.out.tfevents.1660907481.e7a3ae7dcd44.23.0 (deflated 5%)
